In [ ]:
!pip install gradio transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


# **Dataset Processing and Model Implementation**

In [ ]:
import os
import pandas as pd
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
from sklearn.model_selection import train_test_split
import math

# 1. Load data
df = pd.read_csv("merged_dataset.csv")
all_text = ""

# Combine data into a single text
for col in df.select_dtypes(include="object").columns:
    all_text += "\n".join(df[col].dropna().astype(str)) + "\n"

# 2. Split data into training and validation sets
train_text, eval_text = train_test_split(all_text.split("\n"), test_size=0.1, random_state=42)

# Save combined data into text files
with open("train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(train_text))
with open("eval.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(eval_text))

# 3. Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # padding
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 4. Load and organize data into training format
def load_dataset(tokenizer, file_path, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset(tokenizer, "train.txt")
eval_dataset = load_dataset(tokenizer, "eval.txt")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 5. Configure training parameters
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=50,
    save_total_limit=1,
    logging_steps=10,
    logging_first_step=True,
    report_to="none"
)

# 6. Define perplexity calculation function
def compute_perplexity(eval_loss):
    return math.exp(eval_loss)

# 7. Create Trainer
class CustomTrainer(Trainer):
    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        eval_loss = logits.mean()
        perplexity = compute_perplexity(eval_loss)
        return {"eval_loss": eval_loss, "perplexity": perplexity}

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

# 8. Start training
trainer.train()

# 9. Save trained model
model_path1= "./gpt2-finetuned"
trainer.save_model(model_path1)
tokenizer.save_pretrained(model_path1)

# 10. Evaluation and metric calculation
eval_results = trainer.evaluate()
print("\n📊 Evaluation Results:")
print(f"Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,4.805700
10,3.905000
20,3.332800
30,2.898800
40,2.645600
50,2.271600
60,2.308100
70,2.071300
80,1.970000
90,1.872300



📊 Evaluation Results:
Loss: 2.5486
Perplexity: 12.79


# **Example 1**

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model_path1, tokenizer=model_path1)

prompt1 = "My friend"


outputs1 = generator(
    prompt1,
    max_length=30,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    truncation=True
)


print(outputs1[0]["generated_text"].replace("\n", " "))


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


My friend is a great person and I like to The environment provides opportunities for learning about natural systems, including life on Earth's surface. We are dedicated stewards of the oceans' health records The Hubble Space Telescope was designed by George Lucas in 1977 Dark energy has been causing global temperature rise since at least 2002 Asteroid Belt objects orbit around our sun Jovian system that orbits between Jupiter and Saturn with extreme storm activity Orion Nebula consists primarily outgrows from its parent star M1/Exoplanet Oort Cloud System contains more than 1 billion stars within it which make up over 4% (reduced) risk due mainly not only to infrared emissions but also because they contain methane gas That region receives frequent impacts as well known climate events such change Inception directed towards ensuring security worldwide education curriculum based on ancient teachings regarding human origins Alphas focus their attention upon promoting social skills excellen

# **Example 2**

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model_path1, tokenizer=model_path1)

prompt2 = "My book"

outputs2 = generator(
    prompt2,
    max_length=30,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    truncation=True
)
print(outputs2[0]["generated_text"].replace("\n", " "))


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


My book is based on the teachings of The Great Gatsby was released in U.S., Canada and Australia from 1959 to 1989 with an epic storyline that spans over 4 million years ago


# **API  Implementation**

In [ ]:
from transformers import pipeline
import gradio as gr

# Load the model and tokenizer
model =model_path1
generator = pipeline("text-generation", model=model_path1, tokenizer=model_path1)

# Define the text generation function
def generate_text(prompt):
    outputs = generator(
        prompt,
        max_length=20,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        truncation=True
    )
    return outputs[0]["generated_text"].replace("\n", " ")

# Create Gradio interface
interface = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(label="Enter your prompt"),
    outputs=gr.Textbox(label="Generated Text"),
    title="Text Generation API",
    description="Enter a prompt to generate text using a local Hugging Face model.",
    allow_flagging="never",
    live=False
)

interface.launch()


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ae0a880d445d035fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
